<a href="https://colab.research.google.com/github/amonishkumar05/Cross-Market-Analysis/blob/main/crossmarket_analysis(r).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests pandas yfinance

In [2]:
import requests
import pandas as pd

all_data = []

for page in range(1, 3):

    url = f"https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&per_page=250&order=market_cap_desc&page={page}&sparkline=False"

    response = requests.get(url)

    data = response.json()

    all_data.extend(data)

crypto_df = pd.DataFrame(all_data)

print("Total rows:", len(crypto_df))

crypto_df.head()

Total rows: 500


,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://coin-images.coingecko.com/coins/images...,65049.000000,1297883428325,1,1297883428325,3.445608e+10,68212.000000,...,1.999344e+07,2.100000e+07,126080.00,-48.40669,2025-10-06T18:57:42.558Z,67.810000,9.582943e+04,2013-07-06T00:00:00.000Z,None,2026-02-23T05:13:44.691Z
1,ethereum,eth,Ethereum,https://coin-images.coingecko.com/coins/images...,1864.710000,224759695552,2,224759695552,1.586978e+10,1981.090000,...,1.206924e+08,NaN,4946.05,-62.29890,2025-08-24T19:21:03.333Z,0.432979,4.305711e+05,2015-10-20T00:00:00.000Z,"{'times': 37.31101874363215, 'currency': 'btc'...",2026-02-23T05:13:44.730Z
2,tether,usdt,Tether,https://coin-images.coingecko.com/coins/images...,0.999738,183572756249,3,189038160792,5.435527e+10,0.999894,...,1.890912e+11,NaN,1.32,-24.44054,2018-07-24T00:00:00.000Z,0.572521,7.461782e+01,2015-03-02T00:00:00.000Z,None,2026-02-23T05:13:44.333Z
3,ripple,xrp,XRP,https://coin-images.coingecko.com/coins/images...,1.340000,81740191925,4,133940364152,2.252541e+09,1.420000,...,9.998571e+10,1.000000e+11,3.65,-63.19621,2025-07-18T03:40:53.808Z,0.002686,4.985912e+04,2014-05-22T00:00:00.000Z,None,2026-02-23T05:13:42.664Z
4,binancecoin,bnb,BNB,https://coin-images.coingecko.com/coins/images...,589.970000,80293911219,5,80293911219,9.526185e+08,624.450000,...,1.363590e+08,2.000000e+08,1369.99,-56.93622,2025-10-13T08:41:24.131Z,0.039818,1.481578e+06,2017-10-19T00:00:00.000Z,None,2026-02-23T05:13:44.519Z


In [3]:
crypto_df = crypto_df[[
    "id", "symbol", "name", "current_price",
    "market_cap", "market_cap_rank", "total_volume",
    "circulating_supply", "total_supply",
    "ath", "atl", "last_updated"
]]

crypto_df["date"] = pd.to_datetime(crypto_df["last_updated"]).dt.date
crypto_df.drop("last_updated", axis=1, inplace=True)

crypto_df.head()

,id,symbol,name,current_price,market_cap,market_cap_rank,total_volume,circulating_supply,total_supply,ath,atl,date
0,bitcoin,btc,Bitcoin,65049.000000,1297883428325,1,3.445608e+10,1.999344e+07,1.999344e+07,126080.00,67.810000,2026-02-23
1,ethereum,eth,Ethereum,1864.710000,224759695552,2,1.586978e+10,1.206924e+08,1.206924e+08,4946.05,0.432979,2026-02-23
2,tether,usdt,Tether,0.999738,183572756249,3,5.435527e+10,1.836242e+11,1.890912e+11,1.32,0.572521,2026-02-23
3,ripple,xrp,XRP,1.340000,81740191925,4,2.252541e+09,6.101858e+10,9.998571e+10,3.65,0.002686,2026-02-23
4,binancecoin,bnb,BNB,589.970000,80293911219,5,9.526185e+08,1.363590e+08,1.363590e+08,1369.99,0.039818,2026-02-23


In [4]:
top3 = crypto_df.sort_values("market_cap_rank").head(3)

top3_ids = top3["id"].tolist()

top3_ids

['bitcoin', 'ethereum', 'tether']

In [6]:
all_crypto_prices = pd.DataFrame()

for coin in top3_ids:

    url = f"https://api.coingecko.com/api/v3/coins/{coin}/market_chart?vs_currency=usd&days=365"

    response = requests.get(url)

    data = response.json()["prices"]

    temp_df = pd.DataFrame(data, columns=["timestamp", "price_usd"])

    temp_df["date"] = pd.to_datetime(temp_df["timestamp"], unit="ms").dt.date
    temp_df["coin_id"] = coin

    temp_df = temp_df[["coin_id", "date", "price_usd"]]

    all_crypto_prices = pd.concat([all_crypto_prices, temp_df])

all_crypto_prices.head()

,coin_id,date,price_usd
0,bitcoin,2025-02-24,96327.463091
1,bitcoin,2025-02-25,91396.766869
2,bitcoin,2025-02-26,88755.769336
3,bitcoin,2025-02-27,83900.114965
4,bitcoin,2025-02-28,84709.144778


In [7]:
oil_url = "https://raw.githubusercontent.com/datasets/oil-prices/main/data/wti-daily.csv"

oil_df = pd.read_csv(oil_url)

oil_df["date"] = pd.to_datetime(oil_df["Date"]).dt.date

oil_df = oil_df[["date", "Price"]]
oil_df.columns = ["date", "price_usd"]

oil_df = oil_df[oil_df["date"] >= pd.to_datetime("2020-01-01").date()]

oil_df.head()

,date,price_usd
8569,2020-01-02,61.17
8570,2020-01-03,63.00
8571,2020-01-06,63.27
8572,2020-01-07,62.70
8573,2020-01-08,59.65


In [8]:
import yfinance as yf

tickers = ["^GSPC", "^IXIC", "^NSEI"]

stock_df = pd.DataFrame()

for ticker in tickers:

    data = yf.download(ticker, start="2020-01-01", end="2025-09-30")

    data.reset_index(inplace=True)

    data["date"] = pd.to_datetime(data["Date"]).dt.date
    data["ticker"] = ticker

    temp = data[["date", "Open", "High", "Low", "Close", "Volume", "ticker"]]

    temp.columns = ["date", "open", "high", "low", "close", "volume", "ticker"]

    stock_df = pd.concat([stock_df, temp])

stock_df.head()

/tmp/ipython-input-2679789148.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2020-01-01", end="2025-09-30")
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-2679789148.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2020-01-01", end="2025-09-30")
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-2679789148.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2020-01-01", end="2025-09-30")
[*********************100%***********************]  1 of 1 completed


,date,open,high,low,close,volume,ticker
0,2020-01-02,3244.669922,3258.139893,3235.530029,3257.850098,3459930000,^GSPC
1,2020-01-03,3226.360107,3246.149902,3222.340088,3234.850098,3484700000,^GSPC
2,2020-01-06,3217.550049,3246.840088,3214.639893,3246.280029,3702460000,^GSPC
3,2020-01-07,3241.860107,3244.909912,3232.429932,3237.179932,3435910000,^GSPC
4,2020-01-08,3238.590088,3267.070068,3236.669922,3253.050049,3726840000,^GSPC


In [9]:
#sqlite
import sqlite3

conn = sqlite3.connect("cross_market.db")
cursor = conn.cursor()

print("Database connected successfully")


Database connected successfully


In [10]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS cryptocurrencies (
    id VARCHAR(50) PRIMARY KEY,
    symbol VARCHAR(10),
    name VARCHAR(100),
    current_price DECIMAL(18,6),
    market_cap BIGINT,
    market_cap_rank INT,
    total_volume BIGINT,
    circulating_supply DECIMAL(20,6),
    total_supply DECIMAL(20,6),
    ath DECIMAL(18,6),
    atl DECIMAL(18,6),
    date DATE
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS crypto_prices (
    coin_id VARCHAR(50),
    date DATE,
    price_usd DECIMAL(18,6)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS oil_prices (
    date DATE PRIMARY KEY,
    price_usd DECIMAL(18,6)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS stock_prices (
    date DATE,
    open DECIMAL(18,6),
    high DECIMAL(18,6),
    low DECIMAL(18,6),
    close DECIMAL(18,6),
    volume BIGINT,
    ticker VARCHAR(20)
)
""")

conn.commit()

print("Tables created successfully with updated datatypes")

Tables created successfully with updated datatypes


In [11]:
for _, row in crypto_df.iterrows():
    cursor.execute("""
        INSERT INTO cryptocurrencies VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        row["id"],
        row["symbol"],
        row["name"],
        row["current_price"],
        row["market_cap"],
        row["market_cap_rank"],
        row["total_volume"],
        row["circulating_supply"],
        row["total_supply"],
        row["ath"],
        row["atl"],
        row["date"]
    ))

conn.commit()
print("Cryptocurrencies data inserted")

Cryptocurrencies data inserted


In [12]:
for _, row in all_crypto_prices.iterrows():
    cursor.execute("""
        INSERT INTO crypto_prices VALUES (?, ?, ?)
    """, (
        row["coin_id"],
        row["date"],
        row["price_usd"]
    ))

conn.commit()
print("Crypto prices inserted")

Crypto prices inserted


In [13]:
for _, row in oil_df.iterrows():
    cursor.execute("""
        INSERT INTO oil_prices VALUES (?, ?)
    """, (
        row["date"],
        row["price_usd"]
    ))

conn.commit()
print("Oil prices inserted")

Oil prices inserted


In [14]:
for _, row in stock_df.iterrows():
    cursor.execute("""
        INSERT INTO stock_prices VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (row["date"],row["open"],row["high"],row["low"],row["close"],row["volume"],row["ticker"]
    ))

conn.commit()
print("Stock prices inserted")

Stock prices inserted


In [15]:
#for verify the values which we have inserted
pd.read_sql("SELECT COUNT(*) FROM cryptocurrencies", conn)


,COUNT(*)
0,500


In [16]:
#this query is for Top 3 Cryptocurrencies

query1 = """
SELECT id, name, market_cap
FROM cryptocurrencies
ORDER BY market_cap DESC
LIMIT 3
"""

result1 = pd.read_sql(query1, conn)
result1

,id,name,market_cap
0,bitcoin,Bitcoin,1297883428325
1,ethereum,Ethereum,224759695552
2,tether,Tether,183572756249


In [17]:
#this query is for circulating supply

query2 = """
SELECT id, name, circulating_supply, total_supply
FROM cryptocurrencies
WHERE circulating_supply >= 0.9 * total_supply
"""

result2 = pd.read_sql(query2, conn)
result2

,id,name,circulating_supply,total_supply
0,bitcoin,Bitcoin,1.999344e+07,1.999344e+07
1,ethereum,Ethereum,1.206924e+08,1.206924e+08
2,tether,Tether,1.836242e+11,1.890912e+11
3,binancecoin,BNB,1.363590e+08,1.363590e+08
4,usd-coin,USDC,7.442924e+10,7.443290e+10
...,...,...,...,...
264,ankr,Ankr Network,1.000000e+10,1.000000e+10
265,austin-capitals,Austin Capitals,9.786000e+06,9.786000e+06
266,mask-network,Mask Network,1.000000e+08,1.000000e+08
267,singularry,Singularry,9.998975e+08,9.998975e+08


In [18]:
#this query is for coins within 10% of ATH

query3 = """
SELECT id, name, current_price, ath
FROM cryptocurrencies
WHERE current_price >= 0.9 * ath
"""

result3 = pd.read_sql(query3, conn)
result3

,id,name,current_price,ath
0,usds,USDS,0.999580,1.057000
1,ethena-usde,Ethena USDe,0.998819,1.034000
2,usd1-wlfi,USD1,0.999349,1.025000
3,paypal-usd,PayPal USD,0.999967,1.021000
4,tether-gold,Tether Gold,5141.940000,5596.700000
...,...,...,...,...
68,midas-mtbill,Midas mTBILL,1.053000,1.053000
69,sygnum-fiusd-liquidity-fund,Sygnum FIUSD Liquidity Fund,11781.050000,11781.050000
70,resolv-rlp,Resolv Liquidity Provider Token,1.280000,1.350000
71,austin-capitals,Austin Capitals,4.250000,4.380000


In [19]:
#this query is for Average Market Cap Rank

query4 = """
SELECT AVG(market_cap_rank) AS avg_market_cap_rank
FROM cryptocurrencies
WHERE total_volume > 1000000000
"""

result4 = pd.read_sql(query4, conn)
result4

,avg_market_cap_rank
0,6.285714


In [20]:
#this query is for Most Recently Updated Coin

query5 = """
SELECT id, name, date
FROM cryptocurrencies
ORDER BY date DESC
LIMIT 1
"""

result5 = pd.read_sql(query5, conn)
result5

,id,name,date
0,bitcoin,Bitcoin,2026-02-23


In [21]:
query6 = """
SELECT MAX(price_usd) AS highest_price
FROM crypto_prices
WHERE coin_id = 'bitcoin'
"""

result6 = pd.read_sql(query6, conn)
result6

,highest_price
0,124773.508231


In [22]:
query7 = """
SELECT AVG(price_usd) AS average_price
FROM crypto_prices
WHERE coin_id = 'ethereum'
"""

result7 = pd.read_sql(query7, conn)
result7

,average_price
0,2999.82657


In [23]:
query8 = """
SELECT date, price_usd
FROM crypto_prices
WHERE coin_id = 'bitcoin'
AND strftime('%Y-%m', date) = '2025-01'
"""

result8 = pd.read_sql(query8, conn)
result8

,date,price_usd


In [24]:
#oil_price queries
#Highest Oil Price in Last 5 Years
query_oil1 = """
SELECT MAX(price_usd) AS highest_oil_price
FROM oil_prices
WHERE date >= date('now','-5 years')
"""

pd.read_sql(query_oil1, conn)

,highest_oil_price
0,123.64


In [25]:
#Average Oil Price Per Year
query_oil2 = """
SELECT strftime('%Y', date) AS year,
       AVG(price_usd) AS avg_price
FROM oil_prices
GROUP BY year
ORDER BY year
"""

pd.read_sql(query_oil2, conn)

,year,avg_price
0,2020,39.160437
1,2021,68.135100
2,2022,94.902869
3,2023,77.576532
4,2024,76.632240
5,2025,65.388105
6,2026,61.238065


In [26]:
#Oil Prices During COVID Crash (March–April 2020)
query_oil3 = """
SELECT date, price_usd
FROM oil_prices
WHERE date BETWEEN '2020-03-01' AND '2020-04-30'
"""

pd.read_sql(query_oil3, conn)

,date,price_usd
0,2020-03-02,46.78
1,2020-03-03,47.27
2,2020-03-04,46.78
3,2020-03-05,45.90
4,2020-03-06,41.14
5,2020-03-09,31.05
6,2020-03-10,34.47
7,2020-03-11,33.13
8,2020-03-12,31.56
9,2020-03-13,31.72


In [27]:
#Lowest Oil Price in Last 10 Years
query_oil4 = """
SELECT MIN(price_usd) AS lowest_oil_price
FROM oil_prices
"""

pd.read_sql(query_oil4, conn)

,lowest_oil_price
0,-36.98


In [28]:
#Volatility of Oil Prices (Max-Min Per Year)
query_oil5 = """
SELECT strftime('%Y', date) AS year,
       MAX(price_usd) - MIN(price_usd) AS volatility
FROM oil_prices
GROUP BY year
ORDER BY year
"""

pd.read_sql(query_oil5, conn)

,year,volatility
0,2020,100.25
1,2021,38.17
2,2022,52.59
3,2023,27.06
4,2024,20.96
5,2025,25.29
6,2026,8.79


In [29]:
#Get All Stock Prices for Given Ticker eg: ^GSPC
query_stock1 = """
SELECT *
FROM stock_prices
WHERE ticker = '^GSPC'
LIMIT 10
"""

pd.read_sql(query_stock1, conn)

,date,open,high,low,close,volume,ticker
0,2020-01-02,3244.669922,3258.139893,3235.530029,3257.850098,3459930000,^GSPC
1,2020-01-03,3226.360107,3246.149902,3222.340088,3234.850098,3484700000,^GSPC
2,2020-01-06,3217.550049,3246.840088,3214.639893,3246.280029,3702460000,^GSPC
3,2020-01-07,3241.860107,3244.909912,3232.429932,3237.179932,3435910000,^GSPC
4,2020-01-08,3238.590088,3267.070068,3236.669922,3253.050049,3726840000,^GSPC
5,2020-01-09,3266.030029,3275.580078,3263.669922,3274.699951,3641230000,^GSPC
6,2020-01-10,3281.810059,3282.989990,3260.860107,3265.350098,3214580000,^GSPC
7,2020-01-13,3271.129883,3288.129883,3268.429932,3288.129883,3459390000,^GSPC
8,2020-01-14,3285.350098,3294.250000,3277.189941,3283.149902,3687620000,^GSPC
9,2020-01-15,3282.270020,3298.659912,3280.689941,3289.290039,3721490000,^GSPC


In [30]:
#Highest Closing Price for NASDAQ (^IXIC)
query_stock2 = """
SELECT MAX(close) AS highest_close
FROM stock_prices
WHERE ticker = '^IXIC'
"""

pd.read_sql(query_stock2, conn)

,highest_close
0,22788.980469


In [31]:
#Top 5 Days With Highest Price Difference (S&P 500)
query_stock3 = """
SELECT date, (high - low) AS price_difference
FROM stock_prices
WHERE ticker = '^GSPC'
ORDER BY price_difference DESC
LIMIT 5
"""

pd.read_sql(query_stock3, conn)

,date,price_difference
0,2025-04-09,532.909668
1,2025-04-07,411.529785
2,2025-04-08,357.050293
3,2025-04-10,237.879883
4,2025-04-04,222.240234


In [32]:
#Monthly Average Closing Price for Each Ticker
query_stock4 = """
SELECT ticker,
       strftime('%Y-%m', date) AS month,
       AVG(close) AS avg_close
FROM stock_prices
GROUP BY ticker, month
ORDER BY ticker, month
"""

pd.read_sql(query_stock4, conn)

,ticker,month,avg_close
0,^GSPC,2020-01,3278.202858
1,^GSPC,2020-02,3277.314183
2,^GSPC,2020-03,2652.393632
3,^GSPC,2020-04,2761.975226
4,^GSPC,2020-05,2919.608484
...,...,...,...
202,^NSEI,2025-05,24676.440290
203,^NSEI,2025-06,25003.397693
204,^NSEI,2025-07,25164.891474
205,^NSEI,2025-08,24698.102796


In [33]:
#Average Trading Volume of NSEI in 2024
query_stock5 = """
SELECT AVG(volume) AS avg_volume_2024
FROM stock_prices
WHERE ticker = '^NSEI'
AND strftime('%Y', date) = '2024'
"""

pd.read_sql(query_stock5, conn)

,avg_volume_2024
0,313560.162602


In [34]:
#Join Queries

#Compare Bitcoin vs Oil Average Price in 2025
query_join1 = """
SELECT
    AVG(c.price_usd) AS avg_bitcoin_price,
    AVG(o.price_usd) AS avg_oil_price
FROM crypto_prices c
JOIN oil_prices o
ON c.date = o.date
WHERE c.coin_id = 'bitcoin'
AND strftime('%Y', c.date) = '2025'
"""

pd.read_sql(query_join1, conn)

,avg_bitcoin_price,avg_oil_price
0,102287.060169,63.981028


In [35]:
#Check if Bitcoin Moves with S&P 500 (Basic Comparison)
query_join2 = """
SELECT
    c.date,
    c.price_usd AS bitcoin_price,
    s.close AS sp500_close
FROM crypto_prices c
JOIN stock_prices s
ON c.date = s.date
WHERE c.coin_id = 'bitcoin'
AND s.ticker = '^GSPC'
ORDER BY c.date
LIMIT 20
"""

pd.read_sql(query_join2, conn)

,date,bitcoin_price,sp500_close
0,2025-02-24,96327.463091,5983.250000
1,2025-02-25,91396.766869,5955.250000
2,2025-02-26,88755.769336,5956.060059
3,2025-02-27,83900.114965,5861.569824
4,2025-02-28,84709.144778,5954.500000
5,2025-03-03,94261.532865,5849.720215
6,2025-03-04,86124.714187,5778.149902
7,2025-03-05,87310.805311,5842.629883
8,2025-03-06,90604.080985,5738.520020
9,2025-03-07,90001.400876,5770.200195


In [36]:
#Compare Ethereum and NASDAQ Daily Prices for 2025
query_join3 = """
SELECT
    c.date,
    c.price_usd AS ethereum_price,
    s.close AS nasdaq_close
FROM crypto_prices c
JOIN stock_prices s
ON c.date = s.date
WHERE c.coin_id = 'ethereum'
AND s.ticker = '^IXIC'
AND strftime('%Y', c.date) = '2025'
ORDER BY c.date
"""

pd.read_sql(query_join3, conn)

,date,ethereum_price,nasdaq_close
0,2025-02-24,2827.184408,19286.929688
1,2025-02-25,2503.103420,19026.390625
2,2025-02-26,2495.315452,19075.259766
3,2025-02-27,2325.853935,18544.419922
4,2025-02-28,2305.322938,18847.279297
...,...,...,...
146,2025-09-23,4199.951774,22573.470703
147,2025-09-24,4166.190550,22497.859375
148,2025-09-25,4148.656828,22384.699219
149,2025-09-26,3863.063288,22484.070312


In [37]:
#Days When Oil Price Spiked & Bitcoin Price Change
query_join4 = """
SELECT
    o.date,
    o.price_usd AS oil_price,
    c.price_usd AS bitcoin_price
FROM oil_prices o
JOIN crypto_prices c
ON o.date = c.date
WHERE c.coin_id = 'bitcoin'
AND o.price_usd > 100
ORDER BY o.date
"""

pd.read_sql(query_join4, conn)

,date,oil_price,bitcoin_price


In [38]:
#Compare Top 3 Coins vs NIFTY (^NSEI)
query_join5 = """
SELECT
    c.date,
    c.coin_id,
    c.price_usd AS crypto_price,
    s.close AS nifty_close
FROM crypto_prices c
JOIN stock_prices s
ON c.date = s.date
WHERE s.ticker = '^NSEI'
ORDER BY c.date
LIMIT 30
"""

pd.read_sql(query_join5, conn)

,date,coin_id,crypto_price,nifty_close
0,2025-02-24,bitcoin,96327.463091,22553.349609
1,2025-02-24,ethereum,2827.184408,22553.349609
2,2025-02-24,tether,0.999983,22553.349609
3,2025-02-25,bitcoin,91396.766869,22547.550781
4,2025-02-25,ethereum,2503.103420,22547.550781
5,2025-02-25,tether,0.999726,22547.550781
6,2025-02-27,bitcoin,83900.114965,22545.050781
7,2025-02-27,ethereum,2325.853935,22545.050781
8,2025-02-27,tether,0.998532,22545.050781
9,2025-02-28,bitcoin,84709.144778,22124.699219


In [39]:
#Compare S&P 500 with Crude Oil Same Dates
query_join6 = """
SELECT
    s.date,
    s.close AS sp500_close,
    o.price_usd AS oil_price
FROM stock_prices s
JOIN oil_prices o
ON s.date = o.date
WHERE s.ticker = '^GSPC'
ORDER BY s.date
LIMIT 30
"""

pd.read_sql(query_join6, conn)

,date,sp500_close,oil_price
0,2020-01-02,3257.850098,61.17
1,2020-01-03,3234.850098,63.00
2,2020-01-06,3246.280029,63.27
3,2020-01-07,3237.179932,62.70
4,2020-01-08,3253.050049,59.65
5,2020-01-09,3274.699951,59.56
6,2020-01-10,3265.350098,59.02
7,2020-01-13,3288.129883,58.17
8,2020-01-14,3283.149902,58.34
9,2020-01-15,3289.290039,57.86


In [40]:
#Bitcoin vs Crude Oil (Same Date Comparison)
query_join7 = """
SELECT
    c.date,
    c.price_usd AS bitcoin_price,
    o.price_usd AS oil_price
FROM crypto_prices c
JOIN oil_prices o
ON c.date = o.date
WHERE c.coin_id = 'bitcoin'
ORDER BY c.date
LIMIT 30
"""

pd.read_sql(query_join7, conn)

,date,bitcoin_price,oil_price
0,2025-02-24,96327.463091,71.06
1,2025-02-25,91396.766869,69.15
2,2025-02-26,88755.769336,68.87
3,2025-02-27,83900.114965,70.62
4,2025-02-28,84709.144778,69.97
5,2025-03-03,94261.532865,68.63
6,2025-03-04,86124.714187,68.47
7,2025-03-05,87310.805311,66.58
8,2025-03-06,90604.080985,66.62
9,2025-03-07,90001.400876,67.29


In [41]:
#NASDAQ vs Ethereum Trend
query_join8 = """
SELECT
    c.date,
    c.price_usd AS ethereum_price,
    s.close AS nasdaq_close
FROM crypto_prices c
JOIN stock_prices s
ON c.date = s.date
WHERE c.coin_id = 'ethereum'
AND s.ticker = '^IXIC'
ORDER BY c.date
LIMIT 30
"""

pd.read_sql(query_join8, conn)

,date,ethereum_price,nasdaq_close
0,2025-02-24,2827.184408,19286.929688
1,2025-02-25,2503.103420,19026.390625
2,2025-02-26,2495.315452,19075.259766
3,2025-02-27,2325.853935,18544.419922
4,2025-02-28,2305.322938,18847.279297
5,2025-03-03,2517.338239,18350.189453
6,2025-03-04,2148.199533,18285.160156
7,2025-03-05,2171.496944,18552.730469
8,2025-03-06,2241.384115,18069.259766
9,2025-03-07,2202.320736,18196.220703


In [42]:
#Join Top 3 Coins with Stock Indices for 2025
query_join9 = """
SELECT
    c.date,
    c.coin_id,
    c.price_usd,
    s.ticker,
    s.close
FROM crypto_prices c
JOIN stock_prices s
ON c.date = s.date
WHERE strftime('%Y', c.date) = '2025'
ORDER BY c.date
LIMIT 50
"""

pd.read_sql(query_join9, conn)

,date,coin_id,price_usd,ticker,close
0,2025-02-24,bitcoin,96327.463091,^GSPC,5983.250000
1,2025-02-24,bitcoin,96327.463091,^IXIC,19286.929688
2,2025-02-24,bitcoin,96327.463091,^NSEI,22553.349609
3,2025-02-24,ethereum,2827.184408,^GSPC,5983.250000
4,2025-02-24,ethereum,2827.184408,^IXIC,19286.929688
5,2025-02-24,ethereum,2827.184408,^NSEI,22553.349609
6,2025-02-24,tether,0.999983,^GSPC,5983.250000
7,2025-02-24,tether,0.999983,^IXIC,19286.929688
8,2025-02-24,tether,0.999983,^NSEI,22553.349609
9,2025-02-25,bitcoin,91396.766869,^GSPC,5955.250000


In [43]:
#Multi-Join: Bitcoin + Oil + S&P 500
query_join10 = """
SELECT
    c.date,
    c.price_usd AS bitcoin_price,
    o.price_usd AS oil_price,
    s.close AS sp500_close
FROM crypto_prices c
JOIN oil_prices o
ON c.date = o.date
JOIN stock_prices s
ON c.date = s.date
WHERE c.coin_id = 'bitcoin'
AND s.ticker = '^GSPC'
ORDER BY c.date
LIMIT 30
"""

pd.read_sql(query_join10, conn)

,date,bitcoin_price,oil_price,sp500_close
0,2025-02-24,96327.463091,71.06,5983.250000
1,2025-02-25,91396.766869,69.15,5955.250000
2,2025-02-26,88755.769336,68.87,5956.060059
3,2025-02-27,83900.114965,70.62,5861.569824
4,2025-02-28,84709.144778,69.97,5954.500000
5,2025-03-03,94261.532865,68.63,5849.720215
6,2025-03-04,86124.714187,68.47,5778.149902
7,2025-03-05,87310.805311,66.58,5842.629883
8,2025-03-06,90604.080985,66.62,5738.520020
9,2025-03-07,90001.400876,67.29,5770.200195


#STREAMLIT APPLICATION

In [44]:
!pip install -q streamlit
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
import subprocess
subprocess.Popen(["./cloudflared-linux-amd64", "tunnel", "--url", "http://localhost:8501"])
!nohup /content/cloudflared-linux-amd64 tunnel --url http://localhost:8501 &

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 77.8 MB/s eta 0:00:00
--2026-02-23 05:26:54--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2026.2.0/cloudflared-linux-amd64 [following]
--2026-02-23 05:26:54--  https://github.com/cloudflare/cloudflared/releases/download/2026.2.0/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/f9298ca8-89c8-41fe-a51f-e24cb2059878?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-02-23T06%3A19%3A22Z&rscd=attachment%3B+filename%3Dclou

In [45]:
!pip install streamlit-option-menu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.3/829.3 kB 9.5 MB/s eta 0:00:00


In [58]:
%%writefile app.py
import streamlit as st
import pandas as pd
import sqlite3
from streamlit_option_menu import option_menu

# Database connection
conn = sqlite3.connect("cross_market.db")

st.title("💰🛢️📈 Cross-Market Analysis")

with st.sidebar:
    selected = option_menu(
        "Main Menu",
        ["Home", "Filters", "Queries"],
        icons=['house', 'gear', 'search'],
        menu_icon="cast",
        default_index=0
    )
# ---------------------------------------------------
# HOME PAGE
# ---------------------------------------------------
if selected == "Home":

    st.header("📊 Cross-Market Financial Analysis Dashboard")

    st.markdown("""
    ### 📌 Project Objective
    This project analyzes the relationship between:
    - Cryptocurrency Market (Bitcoin & Ethereum)
    - Crude Oil Prices
    - Stock Market Indices (S&P 500, NASDAQ, NIFTY)

    The goal is to understand cross-market movements and trends using SQL-based analysis.
    """)

    st.divider()

    st.subheader("📂 Data Sources Used")

    col1, col2, col3 = st.columns(3)

    with col1:
        st.info("Crypto Data\n\n• Bitcoin\n• Ethereum\n• Market Cap\n• Volume")

    with col2:
        st.info("Oil Market\n\n• Historical Oil Prices\n• Yearly Trends\n• Volatility Analysis")

    with col3:
        st.info("Stock Market\n\n• S&P 500\n• NASDAQ\n• NIFTY\n• Daily Close Prices")

    st.divider()

    st.subheader("🛠️ Technologies Used")

    st.write("""
    - Python
    - Pandas
    - SQLite (Relational Database)
    - SQL Queries
    - Streamlit Dashboard
    """)

    st.success("Use the sidebar to explore Filters and SQL Queries.")

# ---------------------------------------------------
# FILTERS PAGE
# ---------------------------------------------------
if selected == "Filters":

    st.header("Market Filters")

    start_date = st.date_input("Start Date")
    end_date = st.date_input("End Date")

    if start_date and end_date:
      query = f"""
      SELECT c.date,
      c.price_usd AS bitcoin_price,o.price_usd AS oil_price,
      s.close AS sp500_close
      FROM crypto_prices c
      LEFT JOIN oil_prices o ON c.date = o.date
      LEFT JOIN stock_prices s
      ON c.date = s.date
      AND s.ticker = '^GSPC'
      WHERE c.coin_id = 'bitcoin'
      AND c.date BETWEEN '{start_date}' AND '{end_date}'
      ORDER BY c.date
      """

df = pd.read_sql(query, conn)
st.subheader("Filtered Market Data")
st.dataframe(df)
st.line_chart(df.set_index("date"))

# ---------------------------------------------------
# QUERIES PAGE
# ---------------------------------------------------
if selected == "Queries":

    st.header("SQL Queries")

    option = st.selectbox(
        "Select a Query",
        (
            # Cryptocurrencies
            "Top 3 Cryptocurrencies by Market Cap",
            "Coins With Circulating Supply > 90%",
            "Coins Within 10% of ATH",
            "Average Market Cap Rank (Volume > 1B)",
            "Most Recently Updated Coin",

            # Crypto Prices
            "Highest Bitcoin Price",
            "Average Ethereum Price",
            "Bitcoin January Trend",

            # Oil Queries
            "Highest Oil Price",
            "Average Oil Price Per Year",
            "Lowest Oil Price",

            # Stock Queries
            "Highest S&P 500 Close",
            "Top 5 Highest Price Difference Days",
            "Monthly Average Close (All Stocks)",

            # Join Queries
            "Bitcoin vs Oil Comparison",
            "Bitcoin vs S&P 500",
            "Ethereum vs NASDAQ"
        )
    )

    # -------------------------------
    # CRYPTOCURRENCIES
    # -------------------------------

    if option == "Top 3 Cryptocurrencies by Market Cap":
        df = pd.read_sql("""
            SELECT id, name, market_cap
            FROM cryptocurrencies
            ORDER BY market_cap DESC
            LIMIT 3
        """, conn)

    elif option == "Coins With Circulating Supply > 90%":
        df = pd.read_sql("""
            SELECT id, name, circulating_supply, total_supply
            FROM cryptocurrencies
            WHERE circulating_supply >= 0.9 * total_supply
        """, conn)

    elif option == "Coins Within 10% of ATH":
        df = pd.read_sql("""
            SELECT id, name, current_price, ath
            FROM cryptocurrencies
            WHERE current_price >= 0.9 * ath
        """, conn)

    elif option == "Average Market Cap Rank (Volume > 1B)":
        df = pd.read_sql("""
            SELECT AVG(market_cap_rank) AS avg_rank
            FROM cryptocurrencies
            WHERE total_volume > 1000000000
        """, conn)

    elif option == "Most Recently Updated Coin":
        df = pd.read_sql("""
            SELECT id, name, date
            FROM cryptocurrencies
            ORDER BY date DESC
            LIMIT 1
        """, conn)

    # -------------------------------
    # CRYPTO PRICES
    # -------------------------------

    elif option == "Highest Bitcoin Price":
        df = pd.read_sql("""
            SELECT MAX(price_usd) AS highest_price
            FROM crypto_prices
            WHERE coin_id = 'bitcoin'
        """, conn)

    elif option == "Average Ethereum Price":
        df = pd.read_sql("""
            SELECT AVG(price_usd) AS avg_price
            FROM crypto_prices
            WHERE coin_id = 'ethereum'
        """, conn)

    elif option == "Bitcoin January Trend":
        df = pd.read_sql("""
            SELECT date, price_usd
            FROM crypto_prices
            WHERE coin_id = 'bitcoin'
            AND strftime('%m', date) = '01'
        """, conn)

    # -------------------------------
    # OIL
    # -------------------------------

    elif option == "Highest Oil Price":
        df = pd.read_sql("""
            SELECT MAX(price_usd) AS highest_price
            FROM oil_prices
        """, conn)

    elif option == "Average Oil Price Per Year":
        df = pd.read_sql("""
            SELECT strftime('%Y', date) AS year,
                   AVG(price_usd) AS avg_price
            FROM oil_prices
            GROUP BY year
            ORDER BY year
        """, conn)

    elif option == "Lowest Oil Price":
        df = pd.read_sql("""
            SELECT MIN(price_usd) AS lowest_price
            FROM oil_prices
        """, conn)

    # -------------------------------
    # STOCK
    # -------------------------------

    elif option == "Highest S&P 500 Close":
        df = pd.read_sql("""
            SELECT MAX(close) AS highest_close
            FROM stock_prices
            WHERE ticker = '^GSPC'
        """, conn)

    elif option == "Top 5 Highest Price Difference Days":
        df = pd.read_sql("""
            SELECT date, (high - low) AS difference
            FROM stock_prices
            WHERE ticker = '^GSPC'
            ORDER BY difference DESC
            LIMIT 5
        """, conn)

    elif option == "Monthly Average Close (All Stocks)":
        df = pd.read_sql("""
            SELECT ticker,
                   strftime('%Y-%m', date) AS month,
                   AVG(close) AS avg_close
            FROM stock_prices
            GROUP BY ticker, month
            ORDER BY ticker, month
        """, conn)

    # -------------------------------
    # JOIN QUERIES
    # -------------------------------

    elif option == "Bitcoin vs Oil Comparison":
        df = pd.read_sql("""
            SELECT c.date,
                   c.price_usd AS bitcoin_price,
                   o.price_usd AS oil_price
            FROM crypto_prices c
            JOIN oil_prices o ON c.date = o.date
            WHERE c.coin_id = 'bitcoin'
            LIMIT 30
        """, conn)

    elif option == "Bitcoin vs S&P 500":
        df = pd.read_sql("""
            SELECT c.date,
                   c.price_usd AS bitcoin_price,
                   s.close AS sp500_close
            FROM crypto_prices c
            JOIN stock_prices s ON c.date = s.date
            WHERE c.coin_id = 'bitcoin'
            AND s.ticker = '^GSPC'
            LIMIT 30
        """, conn)

    elif option == "Ethereum vs NASDAQ":
        df = pd.read_sql("""
            SELECT c.date,
                   c.price_usd AS eth_price,
                   s.close AS nasdaq_close
            FROM crypto_prices c
            JOIN stock_prices s ON c.date = s.date
            WHERE c.coin_id = 'ethereum'
            AND s.ticker = '^IXIC'
            LIMIT 30
        """, conn)

    st.dataframe(df)


Overwriting app.py


In [59]:
!streamlit run /content/app.py &>/content/logs.txt &

In [60]:
!grep -o 'https://.*\.trycloudflare.com' nohup.out | head -n 1 | xargs -I {} echo "Your tunnel url {}"

Your tunnel url https://tried-civic-mortgage-contracting.trycloudflare.com


In [49]:
!ls

app.py			 cross_market.db  nohup.out
cloudflared-linux-amd64  logs.txt	  sample_data
